## Annotations for fun.

In this notebook we demonstrate how to extract annotations from the DM-EFD using [aioinflux](https://aioinflux.readthedocs.io/en/stable/index.html), a Python client for InfluxDB.

This is complementaty to the [Chronograf](https://test-chronograf-efd.lsst.codes) interface which we use for time-series visualization.

In addition to `aioinflux`, you'll need to install `pandas` to run this notebook.

In [ ]:
import aioinflux
import getpass
import pandas as pd

In [ ]:
import ipywidgets as widgets
from IPython.display import display

We'll access the DM-EFD instance deployed at the AuxTel lab in Tucson. You need to be on site or connected to the NOAO VPN. 

If you are familiar with the AuxTel lab environment, you might be able to authenticate using the generic `saluser`. Ping me at Slack (`@afausti`) if you have any problem.

In [ ]:
username = "saluser"
password = getpass.getpass(f"Password for {username}:")

The following configures the `aioinflux` Python client to connect to the DM-EFD InfluxDB instance. 

In [ ]:
client = aioinflux.InfluxDBClient(host='test-influxdb-efd.lsst.codes', 
                                  port='443', 
                                  ssl=True, 
                                  username=username, 
                                  password=password,
                                  db='efd')

We can configure the output to be a Pandas dataframe, which is very convenient for data analysis.

In [ ]:
client.output = 'dataframe'

## Listing tags of annotations
Topics are mapped to [InfluxDB measurements](https://docs.influxdata.com/influxdb/v1.7/concepts). The following query simply lists all tags on annotations in the `chronograf` database:

In [ ]:
tag_names = await client.query('SHOW TAG KEYS ON "chronograf" FROM "annotations"')
tag_names

Now get all the values for the specific tag `Observer`.

In [ ]:
tag_values = await client.query('SHOW TAG VALUES ON "chronograf" from "annotations" WITH KEY = "Observer"')
w = widgets.Dropdown(
    options=tag_values['value'],
    description='Number:',
    disabled=False,
)
display(w)

## Querying annotations

In Chronograf, you can annotate the time-series to mark interesting events or features in the data. These annotations are saved in the `chronograf` database and can also be queried.

In [ ]:
df = await client.query('SELECT "Observer", "start_time", "modified_time_ns", "text", "id" FROM "chronograf"."autogen"."annotations" WHERE "deleted"=false')

This goes through all the annotations and select those associated with the value of the dropdown box above.

In [ ]:
events = {}
for i, r in df.iterrows():
    if r['Observer'] == w.value:
        if i in events:
            events[i].append(r)
        else:
            events[i] = [r,]

Annotations can be modified many times.  Go through all the annotations and print out the text of the most recent modification for each annotation.

In [ ]:
tz = "US/Pacific"

print(f"Events reported by observer: {w.value}")
for i in events:
    t = 0
    for r in events[i]:
        if r['modified_time_ns'] > t:
            r_to_report = r
            t = r['modified_time_ns']
    start_time = pd.Timestamp(r_to_report['start_time'], tz='GMT')
    if i == start_time:  # should use proper date times for comparison
        print(f"At {i.astimezone(tz)} {r_to_report['Observer']} reported that \"{r_to_report['text']}\" happened")
    else:
        print(f"From {start_time.astimezone(tz)} to {i.astimezone(tz)} {r_to_report['Observer']} reported that \"{r_to_report['text']}\" happened")
